# Keras基本例

以下ではKerasのベースとなるコードを置いておく.

追記など歓迎だが基本は公式exampleが豊富なためそちらを参考にする方が良いだろう.

https://github.com/fchollet/keras/tree/master/examples

大きな違いとして自動微分などをTheanoにやらせるかTensorFlowにやらせるか
選ぶことができる.

初期設定だとTheanoが背後で動くこととなる.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

Using Theano backend.


## 使用データ

何かしらデータを用意する.
ここではchainerの例と全く同じものを置く.

時系列を考慮しない場合など、データ順をシャッフル等したのちに、
教師データとテストデータに分割する必要がある.

ここではダミーデータとして乱数を置いた8変数と0,1,2を順番に教師データとして貼り付けたものを置く.

In [2]:
inputCSV = pd.read_csv('sample.csv')
inputCSV.head(10)

,in0,in1,in2,in3,in4,in5,in6,in7,out
0,0.790158,0.154701,0.554273,0.589591,0.652273,0.809005,0.727239,0.722377,0
1,0.966113,0.082956,0.427091,0.313592,0.036717,0.688759,0.747233,0.084873,1
2,0.744551,0.080415,0.534023,0.874443,0.058886,0.941214,0.234062,0.138623,2
3,0.386799,0.339775,0.758028,0.176749,0.545578,0.645455,0.264393,0.762566,0
4,0.000760,0.339114,0.865157,0.730987,0.061821,0.209777,0.090273,0.183841,1
5,0.497581,0.680770,0.914901,0.990791,0.040909,0.355208,0.881591,0.513112,2
6,0.531146,0.372347,0.246750,0.270516,0.120391,0.682658,0.272929,0.861646,0
7,0.475195,0.512204,0.533743,0.152647,0.070142,0.915401,0.297802,0.617966,1
8,0.841105,0.780319,0.212918,0.535849,0.639757,0.296344,0.653013,0.106019,2
9,0.221653,0.749487,0.411809,0.103275,0.610809,0.310699,0.501801,0.186794,0


In [3]:
# データの並びをシャッフル
calcData = inputCSV.reindex(np.random.permutation(inputCSV.index))
calcData.head(10)

,in0,in1,in2,in3,in4,in5,in6,in7,out
32,0.230549,0.208000,0.333284,0.700938,0.251512,0.935525,0.226757,0.561664,2
553,0.206690,0.232268,0.496381,0.560796,0.436243,0.088000,0.301537,0.918334,1
267,0.881435,0.143742,0.050873,0.930970,0.883936,0.578942,0.911516,0.538892,0
116,0.312668,0.449879,0.415017,0.088452,0.680457,0.988966,0.681649,0.788840,2
112,0.881658,0.351754,0.978736,0.993963,0.979555,0.477020,0.313229,0.459503,1
464,0.766460,0.944161,0.802672,0.690536,0.052631,0.412545,0.827863,0.956309,2
56,0.737255,0.931755,0.948117,0.154918,0.497587,0.570544,0.385176,0.124761,2
503,0.197783,0.018473,0.467290,0.575517,0.325930,0.310118,0.761745,0.809378,2
310,0.041252,0.782474,0.847327,0.670386,0.011416,0.096203,0.530901,0.207017,1
316,0.781330,0.025329,0.784068,0.938632,0.351923,0.744282,0.070422,0.265129,1


In [4]:
# 入力データと教師データに分割
pd_y = calcData['out']
pd_x = calcData.drop(['out'], axis=1)

# numpy配列にデータフレームから変換
x_array = pd_x.as_matrix()
y_array = pd_y.as_matrix()

In [5]:
# 正規化し,float32の型に変換
x_scale = scale(x_array).astype(np.float32)

# サンプルの一部をテストデータ用に分割
N = 550
X_train, X_test = np.split(x_scale, [N])

### 教師データのラベリング
変換が必要な点に注意.

In [7]:
batch_size = 128
nb_classes = 3
nb_epoch = 5

In [8]:
y_scale = y_array.astype(np.int32)
Y_train, Y_test = np.split(y_scale, [N])

# 離散の分類問題の場合、ベクトルのラベルをto_categoricalで変換しておく必要がある.
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

## Deep learningのモデル定義

以下では暫定的に多層ニューラルネットである3層NNを用いる.

(ここが基本的に変更する場所である.)

なお、活性化関数にReLUを用いる.

また途中に過学習を防ぎ汎化性能を上げる手法であるdropoutが仕込んである.

In [9]:
model = Sequential()
model.add(Dense(70, input_shape=(8,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(70))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))

### モデルは一度コンパイルする形式を取るタイプ

サマリーが出てちょっと見やすい.

In [10]:
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 70)            630         dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 70)            0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 70)            0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 70)            4970        dropout_1[0][0]                  
___________________________________________________________________________________________

### 学習と結果

fitで学習し,evaluateでtestデータから汎化性能をチェックする.

In [11]:
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 550 samples, validate on 99 samples
Epoch 1/5
550/550 [==============================] - 0s - loss: 1.1647 - acc: 0.3218 - val_loss: 1.1157 - val_acc: 0.2929
Epoch 2/5
550/550 [==============================] - 0s - loss: 1.1051 - acc: 0.3691 - val_loss: 1.1060 - val_acc: 0.3434
Epoch 3/5
550/550 [==============================] - 0s - loss: 1.1072 - acc: 0.3764 - val_loss: 1.0989 - val_acc: 0.3131
Epoch 4/5
550/550 [==============================] - 0s - loss: 1.1042 - acc: 0.3582 - val_loss: 1.0956 - val_acc: 0.3232
Epoch 5/5
550/550 [==============================] - 0s - loss: 1.0853 - acc: 0.3982 - val_loss: 1.0923 - val_acc: 0.3434
Test score: 1.09226830499
Test accuracy: 0.343434344036
